In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import time
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, concatenate, TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Data loading

In [ ]:
# Load the data
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [4]:
# The sentences' ids, which will be processed to become our input and target data.
conv_lines[:5]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']"]

# Data preprocessing

In [ ]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [ ]:
# Create a list of all of the conversations' lines' ids.
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

### Creating question inputs and answer targets

In [7]:
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])
        
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


### Text Cleaning

In [8]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = " ".join(text.split())
    return text

In [9]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [10]:
print("======Original questions and answers=======")
print(questions[2])
print(answers[2])

print("\n======Cleaned questions and answers===")
print(clean_questions[2])
print(clean_answers[2])

======Original questions and answers=======
Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

======Cleaned questions and answers===
not the hacking and gagging and spitting part please
okay then how about we try out some french cuisine saturday night


### Selecting questions and answers with appropriate length (<20 words)

In [11]:
# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))
# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

print(lengths['counts'].describe())

print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

count    443232.000000
mean         10.872094
std          12.215895
min           0.000000
25%           4.000000
50%           7.000000
75%          14.000000
max         555.000000
Name: counts, dtype: float64
16.0
19.0
24.0
32.0
58.0


Let's choose length of sequences to be of maximum 20.

In [ ]:
import os
import zipfile
import numpy as np

# Define parameters
VOCAB_SIZE = 20000  # Size of the vocabulary
EMBEDDING_DIM = 100  # Dimension of word embeddings
MAX_LEN = 20  # Max length of input sentences
NUM_SAMPLES = 60000  # Number of training samples
GLOVE_DIR = 'glove-global-vectors-for-word-representation'
EPOCHS = 40
BATCH_SIZE = 256
GLOVE_URL = "https://nlp.stanford.edu/data/glove.6B.zip"
GLOVE_FILE = "glove.6B.zip"

if not os.path.exists(GLOVE_FILE):
    os.system(f"curl -O {GLOVE_URL}")

if not os.path.exists(GLOVE_DIR):
    os.makedirs(GLOVE_DIR)
    with zipfile.ZipFile(GLOVE_FILE, 'r') as zip_ref:
        zip_ref.extractall(GLOVE_DIR)

embedding_index = {}
glove_path = os.path.join(GLOVE_DIR, f'glove.6B.{EMBEDDING_DIM}d.txt')

with open(glove_path, encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coef = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coef

In [13]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i, question in enumerate(clean_questions):
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))


r = np.random.randint(1,len(short_questions))
for i in range(r, r+3):
    print(short_questions[i])
    print(short_answers[i])
    print()

138335
138335
he is doing the best he can charles
he is the son of the captain for chrissakes you would think he would be able to operate this thing

i doubt very much that one even exists
what are you talking about

what did you do with rennie
nothing i went to her cabin and



In [ ]:
encoder_input_text = short_questions[:NUM_SAMPLES]

### Put BOS tag and EOS tag for decoder input

In [ ]:
def tagger(input_text):
  bos = "<BOS> "
  eos = " <EOS>"
  final_target = [bos + text + eos for text in input_text] 
  return final_target

short_answers = short_answers[:NUM_SAMPLES]
decoder_input_text = tagger(short_answers)
decoder_input_text[:5]

['<BOS> not the hacking and gagging and spitting part please <EOS>',
 '<BOS> okay then how about we try out some french cuisine saturday night <EOS>',
 '<BOS> forget it <EOS>',
 '<BOS> let me see what i can do <EOS>',
 '<BOS> right see you are ready for the quiz <EOS>']

In [16]:
print(len(encoder_input_text))
print(len(decoder_input_text))

60000
60000


### Make Vocabulary (VOCAB_SIZE)

In [17]:
tokenizer = Tokenizer(num_words= VOCAB_SIZE, filters='')

def vocab_creater(text_lists, VOCAB_SIZE):  
  tokenizer.fit_on_texts(text_lists)
  dictionary = tokenizer.word_index
  
  word2idx = {}
  idx2word = {}
  for k, v in dictionary.items():
      if v < VOCAB_SIZE:
          word2idx[k] = v
          idx2word[v] = k
      if v >= VOCAB_SIZE-1:
          continue
          
  return word2idx, idx2word

word2idx, idx2word = vocab_creater(text_lists=encoder_input_text+decoder_input_text, VOCAB_SIZE=VOCAB_SIZE)

#print first few key/value pairs
word2idx_first5pairs = {k: word2idx[k] for k in list(word2idx)[:5]}
print('word2idx: ', word2idx_first5pairs)

idx2word_first5pairs = {k: idx2word[k] for k in list(idx2word)[:5]}
print('\nidx2word: ', idx2word_first5pairs)

# Check the length of the dictionaries.
print('word2idx length', len(word2idx))
print('idx2word length', len(idx2word))

word2idx:  {'<bos>': 1, '<eos>': 2, 'you': 3, 'i': 4, 'the': 5}

idx2word:  {1: '<bos>', 2: '<eos>', 3: 'you', 4: 'i', 5: 'the'}
word2idx length 19999
idx2word length 19999


### Tokenize Bag of words to Bag of IDs

In [ ]:
def text2seq(tokenizer, encoder_text, decoder_text, VOCAB_SIZE):
  encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
  decoder_sequences = tokenizer.texts_to_sequences(decoder_text)
  
  return encoder_sequences, decoder_sequences

encoder_sequences, decoder_sequences = text2seq(tokenizer, encoder_input_text, decoder_input_text, VOCAB_SIZE)
print('encoder_sequences:\n', encoder_sequences[:5])
print('\ndecoder_sequences:\n', decoder_sequences[:5])

encoder_sequences:
 [[51, 4, 136, 19, 35, 293, 36, 17814, 54, 12, 6, 101, 36, 3], [7, 5, 9560, 18, 7938, 18, 6088, 393, 139], [3, 13, 465, 16, 48, 12, 6, 45, 941, 12, 6, 28, 159, 164], [1551, 54, 124, 19, 77, 147, 3094, 10, 1488], [17815, 991, 17816, 25, 6, 29, 354]]

decoder_sequences:
 [[1, 7, 5, 9560, 18, 7938, 18, 6088, 393, 139, 2], [1, 101, 86, 42, 37, 19, 221, 48, 83, 1101, 1996, 150, 2], [1, 288, 9, 2], [1, 122, 16, 69, 14, 4, 52, 11, 2], [1, 55, 69, 3, 13, 356, 26, 5, 10775, 2]]


### Padding (MAX_LEN)

In [19]:
def padding(encoder_sequences, decoder_sequences, MAX_LEN):
  
  encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
  decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
  
  return encoder_input_data, decoder_input_data

encoder_input_data, decoder_input_data = padding(encoder_sequences, decoder_sequences, MAX_LEN)
print('encoder_input_data:\n', encoder_input_data)
print('\ndecoder_input_data:\n', decoder_input_data)

encoder_input_data:
 [[  51    4  136 ...    0    0    0]
 [   7    5 9560 ...    0    0    0]
 [   3   13  465 ...    0    0    0]
 ...
 [ 603 3243 2132 ...    0    0    0]
 [ 609   22    6 ...    0    0    0]
 [  40   22   33 ...    0    0    0]]

decoder_input_data:
 [[  1   7   5 ...   0   0   0]
 [  1 101  86 ...   0   0   0]
 [  1 288   9 ...   0   0   0]
 ...
 [  1  68 113 ...   0   0   0]
 [  1  40  22 ...   0   0   0]
 [  1  40  22 ...   0   0   0]]


### Reshape the Data to neural network shape

In [20]:
num_samples = len(encoder_sequences)

def decoder_output_creator(decoder_input_data, num_samples, MAX_LEN, VOCAB_SIZE):
  decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype="float32")
  for i, seqs in enumerate(decoder_input_data):
      for t, seq in enumerate(seqs):
          if t > 0:                
                decoder_output_data[i][t][seq] = 1.  #decoder_output_data[i, t, seq] = 1.
  
  return decoder_output_data

decoder_output_data = decoder_output_creator(decoder_input_data, num_samples, MAX_LEN, VOCAB_SIZE)

In [21]:
print (encoder_input_data.shape)
print (decoder_input_data.shape)
print (decoder_output_data.shape)

(60000, 20)
(60000, 20)
(60000, 20, 20000)


### Word Embedding (EMBEDDING_DIM)

We use Pretraind Word2Vec Model from Glove. We can create embedding layer with Glove with 3 steps:
1. Call Glove file
1. Create Embedding Matrix from our Vocabulary
1. Create Embedding Layer

In [22]:
# Call Glove file
def glove_100d_dictionary(glove_dir):
  embeddings_index = {}
  f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
  for line in f:
      values = line.split()
      word = values[0]
      coefs = np.asarray(values[1:], dtype='float32')
      embeddings_index[word] = coefs
  f.close()
  return embeddings_index

embeddings_index = glove_100d_dictionary(GLOVE_DIR)
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [23]:
#Create Embedding Matrix from our Vocabulary
def embedding_matrix_creater(max_words, embedding_dimension):
  embedding_matrix = np.zeros((max_words, embedding_dimension))  #np.zeros((len(word2idx) + 1, embedding_dimension))
  for word, i in word2idx.items():
        if i < max_words:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
  return embedding_matrix

embedding_matrix = embedding_matrix_creater(VOCAB_SIZE, EMBEDDING_DIM)
print(embedding_matrix.shape)
print((len(word2idx) + 1))

(20000, 100)
20000


In [24]:
#Create Embedding Layer
def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
  
  embedding_layer = Embedding(input_dim = VOCAB_SIZE, 
                              output_dim = EMBEDDING_DIM,
                              input_length = MAX_LEN,
                              weights = [embedding_matrix],
                              trainable = False)
  return embedding_layer

embedding_layer = embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix)

embedding_layer2 = embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, None, embedding_matrix)

###  Split data for training validation & testing

In [25]:
def data_spliter(encoder_input_data, decoder_input_data, test_size1=0.2, test_size2=0.3):  
  en_train, en_test, de_train, de_test = train_test_split(encoder_input_data, decoder_input_data, test_size=test_size1)
  en_train, en_val, de_train, de_val = train_test_split(en_train, de_train, test_size=test_size2)
  
  return en_train, en_val, en_test, de_train, de_val, de_test

en_train, en_val, en_test, de_train, de_val, de_test = data_spliter(encoder_input_data, decoder_input_data)

# Create Seq2Seq Neural Network Architecture

Seq2Seq is a type of Encoder-Decoder model using RNN. It can be used as a model for machine interaction and machine translation. By learning a large number of sequence pairs, this model generates one from the other. More kindly explained, the definition of Seq2Seq is below:
* Input: Text Data
* Output: Text Data as well

Below is our Seq2Seq Neural Network Architecture using LSTM.

In [26]:
def build_seq2seq_model(HIDDEN_DIM=300):
    #set up the encoder
    encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    encoder_embedding = embedding_layer(encoder_inputs)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    encoder_states = [state_h, state_c]
    
    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
    decoder_embedding = embedding_layer(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states)
    
    decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(decoder_dense)(decoder_outputs)
    
    #create seq2seq model
    model = Model([encoder_inputs, decoder_inputs], outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc']) #sparse_categorical_crossentropy as labels in a single integer array
    
    #create encoder model
    encoder_model = Model(encoder_inputs, encoder_states)
    
    #Create sampling/decoder model
    decoder_state_input_h  = Input(shape=(HIDDEN_DIM,))
    decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_embedding2= embedding_layer(decoder_inputs)
    decoder_outputs2, state_h2, state_c2 = decoder_LSTM(decoder_embedding2, initial_state=decoder_states_inputs)
    decoder_states2 = [state_h2, state_c2]
    decoder_outputs2 = decoder_dense(decoder_outputs2)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)
    
    return model, encoder_model, decoder_model

def build_seq2seq_model2(HIDDEN_DIM=300):
    #set up the encoder
    encoder_inputs = Input(shape=(None, ), dtype='int32',)
    encoder_embedding = embedding_layer2(encoder_inputs)
    encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    encoder_states = [state_h, state_c]
    
    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, ), dtype='int32',)
    decoder_embedding = embedding_layer2(decoder_inputs)
    decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states)
    
    decoder_dense = Dense(VOCAB_SIZE, activation='softmax')
    outputs = TimeDistributed(decoder_dense)(decoder_outputs)
    
    #create seq2seq model
    model = Model([encoder_inputs, decoder_inputs], outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc']) #sparse_categorical_crossentropy as labels in a single integer array
    
    #create encoder model
    encoder_model = Model(encoder_inputs, encoder_states)
    
    #Create sampling/decoder model
    decoder_state_input_h  = Input(shape=(HIDDEN_DIM,))
    decoder_state_input_c = Input(shape=(HIDDEN_DIM,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_embedding2= embedding_layer2(decoder_inputs)
    decoder_outputs2, state_h2, state_c2 = decoder_LSTM(decoder_embedding2, initial_state=decoder_states_inputs)
    decoder_states2 = [state_h2, state_c2]
    decoder_outputs2 = decoder_dense(decoder_outputs2)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)
    
    return model, encoder_model, decoder_model

model, encoder_model, decoder_model = build_seq2seq_model2(HIDDEN_DIM=300)
model.summary()

# model, encoder_model, decoder_model = build_seq2seq_model(HIDDEN_DIM=300)
# model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    2000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 300), (None, 481200      embedding_2[0][0]          

In [27]:
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 100)         2000000   
_________________________________________________________________
lstm_1 (LSTM)                [(None, 300), (None, 300) 481200    
Total params: 2,481,200
Trainable params: 481,200
Non-trainable params: 2,000,000
_________________________________________________________________


In [28]:
decoder_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    2000000     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 300)          0                                            
____________________________________________________________________________________________

# Train the model

In [ ]:
mcp = ModelCheckpoint('best_model.hdf5', save_best_only=True, monitor='val_loss', mode='min')

model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.05,
          callbacks= [mcp]
         )

Train on 57000 samples, validate on 3000 samples
Epoch 1/40
57000/57000 [==============================] - 180s 3ms/step - loss: 2.1462 - acc: 0.6566 - val_loss: 1.0507 - val_acc: 0.8236
Epoch 2/40
57000/57000 [==============================] - 150s 3ms/step - loss: 0.6717 - acc: 0.8718 - val_loss: 0.4757 - val_acc: 0.8990
Epoch 3/40
57000/57000 [==============================] - 150s 3ms/step - loss: 0.3677 - acc: 0.9090 - val_loss: 0.3614 - val_acc: 0.9135
Epoch 4/40
57000/57000 [==============================] - 150s 3ms/step - loss: 0.2687 - acc: 0.9201 - val_loss: 0.2997 - val_acc: 0.9194
Epoch 5/40
57000/57000 [==============================] - 150s 3ms/step - loss: 0.2170 - acc: 0.9258 - val_loss: 0.2687 - val_acc: 0.9226
Epoch 6/40
57000/57000 [==============================] - 150s 3ms/step - loss: 0.1835 - acc: 0.9293 - val_loss: 0.2483 - val_acc: 0.9247
Epoch 7/40
57000/57000 [==============================] - 150s 3ms/step - loss: 0.1581 - acc: 0.9317 - val_loss: 0.2307 - v

# Predictions

In [30]:
def translate_sentence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word2idx['<bos>']
    eos = word2idx['<eos>']
    output_sentence = []

    for _ in range(MAX_LEN):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)        
        # Sample a token
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]  # Update states

    return ' '.join(output_sentence)

for index in range(10):
    i = np.random.randint(1, len(encoder_input_data))
    input_seq = encoder_input_data[i:i+1]
    translation = translate_sentence(input_seq)
    print('-')
    print('Input:', short_answers[i])
    print('Response:', translation)

-
Input: i am fine you rest
Response: there's there's there's
-
Input: i will wait here if you do not mind
Response: there's there's
-
Input: oh huh
Response: let's let's let's
-
Input: robert i
Response: there's uhhuh
-
Input: you are on the grift same as me
Response: there's there's
-
Input: how are you doing
Response: who's who's
-
Input: i am going to tell him you said that
Response: there's there's there's
-
Input: glass will cut glass mrs langtry do you know where it was purchased
Response: there's let's there's
-
Input: umm three i believe but i am not sure
Response: there's there's
-
Input: i ai not going to jail and i ai not doing that probation thing again
Response: there's there's there's
